# ساخت یک مدل رگرسیون: آماده‌سازی و تجسم داده‌ها

## **رگرسیون خطی برای کدو تنبل - درس دوم**
#### مقدمه

حالا که ابزارهای لازم برای شروع ساخت مدل‌های یادگیری ماشین با استفاده از Tidymodels و Tidyverse را در اختیار دارید، آماده‌اید تا سوالاتی از داده‌های خود بپرسید. هنگام کار با داده‌ها و اعمال راه‌حل‌های یادگیری ماشین، بسیار مهم است که بدانید چگونه سوالات درست بپرسید تا بتوانید به درستی از پتانسیل‌های مجموعه داده خود استفاده کنید.

در این درس، شما یاد خواهید گرفت:

- چگونه داده‌های خود را برای ساخت مدل آماده کنید.

- چگونه از `ggplot2` برای تجسم داده‌ها استفاده کنید.

سوالی که نیاز به پاسخ آن دارید، تعیین می‌کند که از چه نوع الگوریتم‌های یادگیری ماشین استفاده خواهید کرد. همچنین کیفیت پاسخی که دریافت می‌کنید، به شدت به ماهیت داده‌های شما بستگی دارد.

بیایید این موضوع را با انجام یک تمرین عملی بررسی کنیم.

<p >
   <img src="../../images/unruly_data.jpg"
   width="700"/>
   <figcaption>اثر هنری از @allison_horst</figcaption>


## ۱. وارد کردن داده‌های کدو تنبل و فراخوانی Tidyverse

برای تجزیه و تحلیل این درس به بسته‌های زیر نیاز داریم:

-   `tidyverse`: [Tidyverse](https://www.tidyverse.org/) یک [مجموعه از بسته‌های R](https://www.tidyverse.org/packages) است که برای سریع‌تر، آسان‌تر و لذت‌بخش‌تر کردن علم داده طراحی شده است!

می‌توانید آن‌ها را به این صورت نصب کنید:

`install.packages(c("tidyverse"))`

اسکریپت زیر بررسی می‌کند که آیا بسته‌های مورد نیاز برای تکمیل این ماژول را دارید یا خیر و در صورتی که برخی از آن‌ها موجود نباشند، آن‌ها را برای شما نصب می‌کند.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse)

حالا بیایید چند بسته را اجرا کنیم و [داده‌ها](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/data/US-pumpkins.csv) ارائه‌شده برای این درس را بارگذاری کنیم!


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n =50)

یک نگاه سریع با استفاده از `glimpse()` بلافاصله نشان می‌دهد که داده‌ها شامل مقادیر خالی و ترکیبی از رشته‌ها (`chr`) و داده‌های عددی (`dbl`) هستند. ستون `Date` از نوع کاراکتر است و همچنین یک ستون عجیب به نام `Package` وجود دارد که داده‌های آن ترکیبی از مقادیر مانند `sacks`، `bins` و سایر مقادیر است. در واقع، داده‌ها کمی به‌هم‌ریخته هستند 😤.

در حقیقت، خیلی رایج نیست که یک مجموعه داده کاملاً آماده برای استفاده و ساخت یک مدل یادگیری ماشین به شما هدیه داده شود. اما نگران نباشید، در این درس یاد می‌گیرید که چگونه یک مجموعه داده خام را با استفاده از کتابخانه‌های استاندارد R آماده کنید 🧑‍🔧. همچنین تکنیک‌های مختلفی برای تجسم داده‌ها را خواهید آموخت. 📈📊
<br>

> یک یادآوری: عملگر پایپ (`%>%`) عملیات را به صورت منطقی و پشت سر هم انجام می‌دهد و یک شیء را به جلو به یک تابع یا عبارت ارسال می‌کند. می‌توانید عملگر پایپ را به این صورت تصور کنید که در کد شما به معنای "و سپس" است.


## ۲. بررسی داده‌های ناقص

یکی از رایج‌ترین مشکلاتی که دانشمندان داده باید با آن مواجه شوند، داده‌های ناقص یا گم‌شده است. در زبان R، مقادیر گم‌شده یا ناشناخته با یک مقدار خاص به نام `NA` (مخفف Not Available) نمایش داده می‌شوند.

اما چگونه می‌توانیم بفهمیم که یک فریم داده شامل مقادیر گم‌شده است؟
<br>
-   یکی از روش‌های ساده استفاده از تابع پایه R به نام `anyNA` است که اشیاء منطقی `TRUE` یا `FALSE` را برمی‌گرداند.


In [ ]:
pumpkins %>% 
  anyNA()

عالی است، به نظر می‌رسد برخی داده‌ها وجود ندارند! این می‌تواند نقطه‌ی خوبی برای شروع باشد.

-   یک روش دیگر استفاده از تابع `is.na()` است که نشان می‌دهد کدام عناصر ستون به صورت جداگانه با مقدار منطقی `TRUE` گم شده‌اند.


In [ ]:
pumpkins %>% 
  is.na() %>% 
  head(n = 7)

باشه، کار انجام شد اما با یک دیتافریم بزرگ مثل این، بررسی تک‌تک سطرها و ستون‌ها به‌صورت جداگانه غیربهینه و عملاً غیرممکن است 😴.

-   یک روش شهودی‌تر این است که مجموع مقادیر گمشده را برای هر ستون محاسبه کنیم:


In [ ]:
pumpkins %>% 
  is.na() %>% 
  colSums()

خیلی بهتر! داده‌هایی کم هستند، اما شاید برای انجام این کار مهم نباشند. بیایید ببینیم تحلیل بیشتر چه نتایجی به همراه دارد.

> همراه با مجموعه‌های فوق‌العاده‌ای از پکیج‌ها و توابع، R مستندات بسیار خوبی دارد. برای مثال، از `help(colSums)` یا `?colSums` استفاده کنید تا اطلاعات بیشتری درباره این تابع به دست آورید.


## ۳. Dplyr: دستور زبانی برای دستکاری داده‌ها

<p >
   <img src="../../images/dplyr_wrangling.png"
   width="569"/>
   <figcaption>اثر هنری از @allison_horst</figcaption>


<!--![اثر هنری از \@allison_horst](../../../../../../translated_images/dplyr_wrangling.f5f99c64fd4580f1377fee3ea428b6f8fd073845ec0f8409d483cfe148f0984e.fa.png)<br/>اثر هنری از \@allison_horst-->


[`dplyr`](https://dplyr.tidyverse.org/)، یکی از بسته‌های Tidyverse، یک دستور زبان برای دستکاری داده‌ها است که مجموعه‌ای یکپارچه از افعال را ارائه می‌دهد تا به شما در حل رایج‌ترین چالش‌های دستکاری داده‌ها کمک کند. در این بخش، برخی از افعال dplyr را بررسی خواهیم کرد!  
<br>


#### dplyr::select()

تابع `select()` در بسته `dplyr` به شما کمک می‌کند ستون‌هایی را که می‌خواهید نگه دارید یا حذف کنید، انتخاب کنید.

برای اینکه کار با دیتافریم شما آسان‌تر شود، می‌توانید با استفاده از `select()` چند ستون آن را حذف کرده و فقط ستون‌های مورد نیاز خود را نگه دارید.

برای مثال، در این تمرین، تحلیل ما شامل ستون‌های `Package`، `Low Price`، `High Price` و `Date` خواهد بود. بیایید این ستون‌ها را انتخاب کنیم.


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(Package, `Low Price`, `High Price`, Date)


# Print data set
pumpkins %>% 
  slice_head(n = 5)

#### dplyr::mutate()

`mutate()` یک تابع در بسته `dplyr` است که به شما کمک می‌کند ستون‌های جدید ایجاد کنید یا ستون‌های موجود را تغییر دهید، در حالی که ستون‌های قبلی را حفظ می‌کنید.

ساختار کلی `mutate` به این صورت است:

`data %>%   mutate(new_column_name = what_it_contains)`

بیایید با استفاده از ستون `Date` چند عملیات انجام دهیم تا نحوه کار `mutate` را بررسی کنیم:

1. تاریخ‌ها (که در حال حاضر از نوع کاراکتر هستند) را به فرمت ماه تبدیل کنیم (این تاریخ‌ها به سبک آمریکایی هستند، بنابراین فرمت آن‌ها `MM/DD/YYYY` است).

2. ماه را از تاریخ‌ها استخراج کرده و در یک ستون جدید ذخیره کنیم.

در زبان R، بسته [lubridate](https://lubridate.tidyverse.org/) کار با داده‌های تاریخ و زمان را آسان‌تر می‌کند. بنابراین، بیایید از `dplyr::mutate()`، `lubridate::mdy()`، `lubridate::month()` استفاده کنیم و ببینیم چگونه می‌توانیم اهداف بالا را محقق کنیم. همچنین می‌توانیم ستون `Date` را حذف کنیم، زیرا در عملیات بعدی دیگر به آن نیازی نخواهیم داشت.


In [ ]:
# Load lubridate
library(lubridate)

pumpkins <- pumpkins %>% 
  # Convert the Date column to a date object
  mutate(Date = mdy(Date)) %>% 
  # Extract month from Date
  mutate(Month = month(Date)) %>% 
  # Drop Date column
  select(-Date)

# View the first few rows
pumpkins %>% 
  slice_head(n = 7)

هورا! 🤩

حالا بیایید یک ستون جدید به نام `Price` ایجاد کنیم که نشان‌دهنده قیمت متوسط یک کدو تنبل است. سپس میانگین ستون‌های `Low Price` و `High Price` را محاسبه می‌کنیم تا ستون جدید Price را پر کنیم.


In [ ]:
# Create a new column Price
pumpkins <- pumpkins %>% 
  mutate(Price = (`Low Price` + `High Price`)/2)

# View the first few rows of the data
pumpkins %>% 
  slice_head(n = 5)

بله! 💪

"اما صبر کن!" ممکن است بعد از مرور کل مجموعه داده با استفاده از `View(pumpkins)` بگویی، "اینجا چیزی عجیب به نظر می‌رسد!" 🤔

اگر به ستون `Package` نگاه کنی، کدوها در پیکربندی‌های مختلفی فروخته می‌شوند. برخی در اندازه‌های `1 1/9 bushel` فروخته می‌شوند، برخی در اندازه‌های `1/2 bushel`، برخی به ازای هر کدو، برخی به ازای هر پوند، و برخی در جعبه‌های بزرگی با عرض‌های متغیر.

بیایید این را بررسی کنیم:


In [ ]:
# Verify the distinct observations in Package column
pumpkins %>% 
  distinct(Package)

شگفت‌انگیز!👏

به نظر می‌رسد وزن کردن کدو تنبل‌ها به طور یکنواخت بسیار دشوار است، بنابراین بیایید آن‌ها را فیلتر کنیم و فقط کدو تنبل‌هایی را انتخاب کنیم که رشته *bushel* در ستون `Package` دارند و این را در یک داده‌فریم جدید به نام `new_pumpkins` قرار دهیم.


#### dplyr::filter() و stringr::str_detect()

[`dplyr::filter()`](https://dplyr.tidyverse.org/reference/filter.html): زیرمجموعه‌ای از داده‌ها ایجاد می‌کند که فقط شامل **ردیف‌هایی** است که شرایط شما را برآورده می‌کنند. در این مثال، کدوهایی که رشته *bushel* در ستون `Package` آن‌ها وجود دارد.

[stringr::str_detect()](https://stringr.tidyverse.org/reference/str_detect.html): حضور یا عدم حضور یک الگو در یک رشته را تشخیص می‌دهد.

بسته [`stringr`](https://github.com/tidyverse/stringr) توابع ساده‌ای برای عملیات رایج روی رشته‌ها ارائه می‌دهد.


In [ ]:
# Retain only pumpkins with "bushel"
new_pumpkins <- pumpkins %>% 
       filter(str_detect(Package, "bushel"))

# Get the dimensions of the new data
dim(new_pumpkins)

# View a few rows of the new data
new_pumpkins %>% 
  slice_head(n = 5)

می‌توانید ببینید که ما داده‌ها را به حدود ۴۱۵ ردیف محدود کرده‌ایم که شامل کدو تنبل به صورت بوشل است. 🤩  
<br>


#### dplyr::case_when()

**اما صبر کنید! هنوز یک کار دیگر باقی مانده است**

متوجه شدید که مقدار بوشل در هر سطر متفاوت است؟ باید قیمت‌گذاری را نرمال‌سازی کنید تا قیمت را بر اساس هر بوشل نشان دهید، نه بر اساس ۱ ۱/۹ یا ۱/۲ بوشل. وقت آن است که کمی ریاضی انجام دهید تا آن را استاندارد کنید.

ما از تابع [`case_when()`](https://dplyr.tidyverse.org/reference/case_when.html) استفاده خواهیم کرد تا ستون Price را بر اساس برخی شرایط *تغییر دهیم*. `case_when` به شما این امکان را می‌دهد که چندین عبارت `if_else()` را به صورت برداری بنویسید.


In [ ]:
# Convert the price if the Package contains fractional bushel values
new_pumpkins <- new_pumpkins %>% 
  mutate(Price = case_when(
    str_detect(Package, "1 1/9") ~ Price/(1 + 1/9),
    str_detect(Package, "1/2") ~ Price/(1/2),
    TRUE ~ Price))

# View the first few rows of the data
new_pumpkins %>% 
  slice_head(n = 30)

اکنون می‌توانیم قیمت هر واحد را بر اساس اندازه بوشل تحلیل کنیم. با این حال، تمام این بررسی بوشل‌های کدو تنبل نشان می‌دهد که چقدر `مهم` است که `ماهیت داده‌های خود را درک کنید`!

> ✅ طبق گفته [The Spruce Eats](https://www.thespruceeats.com/how-much-is-a-bushel-1389308)، وزن یک بوشل به نوع محصول بستگی دارد، زیرا این یک اندازه‌گیری حجمی است. "برای مثال، یک بوشل گوجه‌فرنگی باید ۵۶ پوند وزن داشته باشد... برگ‌ها و سبزیجات فضای بیشتری را با وزن کمتر اشغال می‌کنند، بنابراین یک بوشل اسفناج فقط ۲۰ پوند است." این موضوع کاملاً پیچیده است! بیایید خود را درگیر تبدیل بوشل به پوند نکنیم و به جای آن قیمت را بر اساس بوشل تعیین کنیم. تمام این بررسی بوشل‌های کدو تنبل نشان می‌دهد که چقدر مهم است که ماهیت داده‌های خود را درک کنید!
>
> ✅ آیا متوجه شدید که کدو تنبل‌هایی که به صورت نیم‌بوشل فروخته می‌شوند بسیار گران هستند؟ می‌توانید دلیل آن را پیدا کنید؟ نکته: کدو تنبل‌های کوچک بسیار گران‌تر از کدو تنبل‌های بزرگ هستند، احتمالاً به این دلیل که تعداد بیشتری از آن‌ها در هر بوشل وجود دارد، با توجه به فضای خالی‌ای که یک کدو تنبل بزرگ و توخالی اشغال می‌کند.


حالا در نهایت، برای ماجراجویی خالص 💁‍♀️، بیایید ستون ماه را به اولین موقعیت منتقل کنیم، یعنی `قبل از` ستون `Package`.

برای تغییر موقعیت ستون‌ها از `dplyr::relocate()` استفاده می‌شود.


In [ ]:
# Create a new data frame new_pumpkins
new_pumpkins <- new_pumpkins %>% 
  relocate(Month, .before = Package)

new_pumpkins %>% 
  slice_head(n = 7)

کارت عالی بود!👌 حالا یک مجموعه داده تمیز و مرتب در اختیار داری که می‌توانی مدل رگرسیون جدیدت را بر اساس آن بسازی!
<br>


## ۴. مصورسازی داده‌ها با ggplot2

<p >
   <img src="../../images/data-visualization.png"
   width="600"/>
   <figcaption>اینفوگرافیک از داسانی مدیپالی</figcaption>


<!--![اینفوگرافیک از داسانی مدیپالی](../../../../../../translated_images/data-visualization.54e56dded7c1a804d00d027543f2881cb32da73aeadda2d4a4f10f3497526114.fa.png){width="600"}-->

یک ضرب‌المثل *حکیمانه* وجود دارد که می‌گوید:

> "یک نمودار ساده اطلاعات بیشتری به ذهن تحلیل‌گر داده می‌رساند تا هر ابزار دیگری." --- جان توکی

یکی از وظایف دانشمند داده این است که کیفیت و ماهیت داده‌هایی که با آن‌ها کار می‌کند را نشان دهد. برای این کار، آن‌ها اغلب مصورسازی‌های جالبی ایجاد می‌کنند، مانند نمودارها، گراف‌ها و چارت‌ها که جنبه‌های مختلف داده را نمایش می‌دهند. به این ترتیب، آن‌ها می‌توانند روابط و شکاف‌هایی را که به‌طور معمول کشف آن‌ها دشوار است، به‌صورت بصری نشان دهند.

مصورسازی‌ها همچنین می‌توانند به تعیین تکنیک یادگیری ماشینی مناسب برای داده‌ها کمک کنند. به‌عنوان مثال، یک نمودار پراکندگی که به نظر می‌رسد از یک خط پیروی می‌کند، نشان می‌دهد که داده‌ها گزینه خوبی برای یک تمرین رگرسیون خطی هستند.

زبان R چندین سیستم برای ایجاد نمودارها ارائه می‌دهد، اما [`ggplot2`](https://ggplot2.tidyverse.org/index.html) یکی از زیباترین و متنوع‌ترین آن‌هاست. `ggplot2` به شما این امکان را می‌دهد که نمودارها را با **ترکیب اجزای مستقل** بسازید.

بیایید با یک نمودار پراکندگی ساده برای ستون‌های Price و Month شروع کنیم.

در این مورد، ابتدا از [`ggplot()`](https://ggplot2.tidyverse.org/reference/ggplot.html) استفاده می‌کنیم، یک مجموعه داده و نگاشت زیبایی‌شناسی (با [`aes()`](https://ggplot2.tidyverse.org/reference/aes.html)) ارائه می‌دهیم و سپس لایه‌هایی (مانند [`geom_point()`](https://ggplot2.tidyverse.org/reference/geom_point.html)) برای نمودارهای پراکندگی اضافه می‌کنیم.


In [ ]:
# Set a theme for the plots
theme_set(theme_light())

# Create a scatter plot
p <- ggplot(data = new_pumpkins, aes(x = Price, y = Month))
p + geom_point()

آیا این نمودار مفید است 🤷؟ آیا چیزی در مورد آن شما را شگفت‌زده می‌کند؟

نه چندان مفید است، زیرا تنها کاری که انجام می‌دهد نمایش داده‌های شما به صورت پراکندگی نقاط در یک ماه مشخص است.  


### **چگونه آن را مفید کنیم؟**

برای نمایش داده‌های مفید در نمودارها، معمولاً باید داده‌ها را به نوعی گروه‌بندی کنید. به عنوان مثال، در مورد ما، پیدا کردن میانگین قیمت کدو تنبل‌ها برای هر ماه می‌تواند بینش بیشتری درباره الگوهای موجود در داده‌های ما ارائه دهد. این ما را به یک مرور دیگر از **dplyr** می‌رساند:

#### `dplyr::group_by() %>% summarize()`

تجمیع گروه‌بندی شده در R به راحتی با استفاده از

`dplyr::group_by() %>% summarize()`

قابل محاسبه است.

-   `dplyr::group_by()` واحد تحلیل را از کل مجموعه داده به گروه‌های جداگانه مانند هر ماه تغییر می‌دهد.

-   `dplyr::summarize()` یک فریم داده جدید ایجاد می‌کند که شامل یک ستون برای هر متغیر گروه‌بندی و یک ستون برای هر آماری خلاصه‌ای است که مشخص کرده‌اید.

به عنوان مثال، می‌توانیم از `dplyr::group_by() %>% summarize()` استفاده کنیم تا کدو تنبل‌ها را بر اساس ستون **ماه** گروه‌بندی کنیم و سپس **میانگین قیمت** را برای هر ماه پیدا کنیم.


In [ ]:
# Find the average price of pumpkins per month
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price))

مختصر و مفید!✨

ویژگی‌های دسته‌بندی‌شده مانند ماه‌ها بهتر است با استفاده از نمودار میله‌ای نمایش داده شوند 📊. لایه‌هایی که مسئول ایجاد نمودارهای میله‌ای هستند عبارتند از `geom_bar()` و `geom_col()`. برای اطلاعات بیشتر به `?geom_bar` مراجعه کنید.

بیایید یکی درست کنیم!


In [ ]:
# Find the average price of pumpkins per month then plot a bar chart
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price)) %>% 
  ggplot(aes(x = Month, y = mean_price)) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("Pumpkin Price")

🤩🤩 این یک مصورسازی داده بسیار مفیدتر است! به نظر می‌رسد که بالاترین قیمت کدو تنبل در ماه‌های سپتامبر و اکتبر اتفاق می‌افتد. آیا این مطابق با انتظارات شماست؟ چرا یا چرا نه؟

تبریک می‌گویم که درس دوم را به پایان رساندید 👏! شما داده‌های خود را برای ساخت مدل آماده کردید و سپس با استفاده از مصورسازی‌ها، بینش‌های بیشتری کشف کردید!



---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما تلاش می‌کنیم دقت را حفظ کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادرستی‌ها باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، توصیه می‌شود از ترجمه حرفه‌ای انسانی استفاده کنید. ما مسئولیتی در قبال سوء تفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
